In [1]:
import numpy as np
import pandas as pd
import collections

In [42]:
#本次尝试使用python实现决策树算法

def entropy(y) -> float:
    """
    param data: A np.array which contains the attribute y
    """
    gross = len(y)
    all_attr = np.unique(y)
    attributes = len(all_attr)
    freq = np.zeros((1,attributes))
    
    for i in range(attributes):
        counts = len(y[y==all_attr[i]])
        freq[0,i] = counts
    
    p = freq / gross
    lnp = np.log(p)
    entropy = -1 * np.sum(p * lnp)
    
    return entropy

def conditional_entropy(feature,label) -> float:
    """
    param feature : an m*1 np.array which contains one feature with different numbers
    param label : an m*1 np.array ,the Y
    """
    entropy_dict = {}
    
    for value in np.unique(feature):
        entropy_dict[value] = label[np.where(feature==value)]
    
    total_len = len(label)
    con_ent = 0.0
    
    for value in entropy_dict.values():
        p = len(value) / total_len * entropy(value)
        con_ent += p
    
    return con_ent

def info_gain(feature,label) -> float:
    """
    param feature : an m*1 np.array which contains one feature with different numbers
    param label : an m*1 np.array ,the Y
    """
    con_ent = conditional_entropy(feature,label)
    ent = entropy(label)
    return ent - con_ent

class DecisionNode(object):
    """
    The node of the decision tree
    """
    def __init__(self,col=-1,data_set=None,labels=None,results=None,tb=None,fb=None):
        self.has_calc_index = []    #已计算过的特征索引
        self.col = col              #待检验的列索引 
        self.data_set = data_set    #节点的待检测数据
        self.labels = labels        #对应当前列匹配的值
        self.results = results      #保存针对当前分支的结果，有值说明为叶子节点
        self.tb = tb                #当前信息增益最高为True时的特征的子树
        self.fb = fb                #当前信息增益最高为False时的特征的子树

def if_split_end(label) -> bool:
    """
    If splited in only one result , then stop
    """
    length = len(label)
    return length == 1

def choose_best_feature(features,label,ignore_index) -> int:
    """
    param features : a m*n np.array,where n is the num of features
    param label : the Y
    param ignore_index : the index list of calculated features
    """
    n = features.shape[1]
    entropy_dict = {}
    
    for i in range(n):
        if i in ignore_index:
            continue
        entropy_dict[i] = info_gain(features[:,i],label)
    
    ret = sorted(entropy_dict.items(),key=lambda x:x[1],reverse=True)
    return ret[0][0]

class DecisionTree(object):
    """
    Build a decision tree
    """
    def __init__(self):
        self.feature_num = 0
        self.tree_root = None
    
    def build_tree(self,node:DecisionNode):
        """
        param node : a DecisionNode class
        """
        if if_split_end(node.labels):
            node.results = node.labels[0]
            return
        
        best_index = choose_best_feature(node.data_set,node.labels,node.has_calc_index)
        node.col = best_index
        
        #由最大信息增益划分
        #左子树
        tb_index = [i for i,value in enumerate(node.data_set) if value[best_index]]
        tb_data_set = node.data_set[tb_index,:]
        tb_label = node.labels[tb_index]
        tb_node = DecisionNode(date_set=tb_data_set,labels=tb_label)
        tb_node.has_calc_index = list(node.has_calc_index)
        tb_node.has_calc_index.append(best_index)
        node.tb = tb_node
        
        #右子树
        fb_index = [i for i,value in enumerate(node.data_set) if not value[best_index]]
        fb_data_set = node.data_set[fb_index,:]
        fb_label = node.labels[fb_index]
        fb_node = DecisionNode(data_set=fb_data_set,labels=fb_label)
        fb_node.has_calc_index = list(node.has_calc_index)
        fb_node.has_calc_index.append(best_index)
        node.fb = fb_node
        
        if tb_index:
            self.build_tree(node.tb)
        if fb_index:
            self.build_tree(node.fb)
    def clear_tree_example_data(self,node:DecisionNode):
        """
        clear training data of the tree
        """
        del node.has_calc_index
        del node.data_set
        del node.labels
        if node.tb:
            self.clear_tree_example_data(node)
        if node.fb:
            self.clear_tree_example_data(node)
    
    def fit(self,X,y):
        """
        param X : input training data X
        param y : input training result y
        """
        self.feature_num = len(X[0,:])
        self.tree_root = DecisionNode(data_set=X,labels=y)
        self.build_tree(self.tree_root)
        self.clear_tree_example_data(self.tree_root)
    
    def _predict(self,data_test,node):
        """
        param data_test : input test set testX
        param node : the DecisionNode rooted from tree_root
        """
        if node.results:
            return node.results
        col = node.col
        if data_test[col]:
            return self._predict(data_test,node.tb)
        else:
            return self._predict(data_test,node.fb)
    
    def predict(self,data_test):
        """
        to predict
        param data_test : test set testX
        """
        return self._predict(data_test,self.tree_root)

In [56]:
[i for i,value in enumerate(a) if value[1]]

[2, 3]

In [36]:
conditional_entropy(a,b)

0.47738562622110958

In [ ]:
np.where()